In [ ]:
import os
import mne_bids
import pandas as pd

os.environ['OMP_NUM_THREADS'] = '1'


bids_root = r'D:\\ds005234\\'
bids_derivatives = bids_root + r'derivatives/'

subject = 'Z201'


bids_path = mne_bids.BIDSPath(subject=subject,
                              root=bids_root)

file_list = pd.read_csv(bids_path.directory.as_posix() + '/sub-' + subject + '_scans.tsv',
                        delimiter="\t", dtype=str)

file_list = file_list.filename

bids_path.update(datatype='meg')

cal = bids_path.directory.as_posix() + '/sub-' + subject + '_acq-calibration_meg.dat'
ctc = bids_path.directory.as_posix() + '/sub-' + subject + '_acq-crosstalk_meg.fif'


for file in file_list:

    bp = mne_bids.get_bids_path_from_fname(bids_path.directory.as_posix() + file)
    
    channels = pd.read_csv(bp.copy().update(extension='.tsv', suffix='channels'),
                           delimiter="\t", dtype=str)
    
    bad_chan = channels[channels.status=='bad'].name.values
    
    bad_chan_neuromag = ' '.join([str(int(i[3:])) for i in bad_chan])
    
    bp_tsss_mc = bp.copy().update(processing = 'tsss+mc', root = bids_derivatives)
    
    bp_tsss_mc.mkdir()
    
    bp_tsss_mc_trans = bp_tsss_mc.copy().update(processing = 'tsss+mc+trans')
        
    os.system("/neuro/bin/util/maxfilter" +
          " -f " + str(bp.fpath) +
          " -o " + str(bp_tsss_mc.fpath) +
          " -ctc " + ctc + 
          " -cal " + cal +
          " -autobad off" + 
          " -movecomp" + 
          " -format float -st 10 -corr 0.900" +
          " -bad " + bad_chan_neuromag + 
          " -v | tee " + str(bp_tsss_mc.fpath)[:-3] + 'log')

    os.system("/neuro/bin/util/maxfilter" +
          " -f " + str(bp_tsss_mc.fpath) +
          " -o " + str(bp_tsss_mc_trans.fpath) +
          " -ctc " + ctc + 
          " -cal " + cal +
          " -autobad off" + 
          " -trans default -frame head -origin 0 0 45" + 
          " -force" +
          " -v | tee " + str(bp_tsss_mc_trans.fpath)[:-3] + 'log')